In [1]:
import pandas as pd
from pandas import *
from pandas.io.json import json_normalize

import json

pd.options.display.max_columns = 50

In [2]:
with open('../incoming/raw.json') as f:
    data = json.load(f)

In [24]:
questions = json_normalize(data, record_path=['questions'], meta=['country', 'name'])
questions['question_shared_id'] = questions.apply(lambda x: x.name % 22, axis=1)
question_params = json_normalize(data, record_path=['questions', 'params'], meta=[['questions', 'id']])
question_params.columns = ['id', 'questions.id']
question_groups = json_normalize(data, record_path='questionParams')
party_positions = json_normalize(data, record_path=['partyPositions', 'answers'],
               meta=['country', ['partyPositions', 'id'], ['partyPositions', 'name'],
                    ['partyPositions', 'party'], ['partyPositions', 'image']])

In [25]:
question_params.id = question_params.id.apply(lambda x: str(x))
question_groups.id = question_groups.id.apply(lambda x: str(x))
qpg = pd.merge(question_params, question_groups, on='id', how='left')
qpg.value = qpg.value.fillna(qpg['id'])

In [26]:
def map_to_groups(x):
    result = {}

    for k in x.params.keys():
        result[qpg[(qpg['questions.id'] == x.id) & (qpg['id'] == k)].values[0][2] \
               .lower().replace(' ', '_')] = x.params[k]
        
    return result

questions.params = questions.apply(map_to_groups, axis=1)

questions = questions.join(questions.params.apply(pd.Series))

In [27]:
full = pd.merge(party_positions, questions, left_on='question_id', right_on='id')

In [28]:
full.head()

,answer,question_id,source,source_desc,source_link,partyPositions.id,country_x,partyPositions.party,partyPositions.name,partyPositions.image,description,id,params,position,value,country_y,name,question_shared_id,economic_liberalisation,environmental_protection,european_integration,law_and_order,liberal_society,restrictive_financial_policy,restrictive_immigration_policy,xy-lc,xy-lr
0,100.0,751,Leitantrag 3 des 44. ordentlichen Bundespartei...,"""Gerade der Europäischen Union stehen die Mögl...",https://www.spoe.at/wp-content/uploads/sites/7...,332,AT,Social Democratic Party of Austria,SPÖ,http://files2.societly.com.s3-website.eu-centr...,,751,"{u'law_and_order': u'0', u'restrictive_financi...",None,Social programmes should be maintained even at...,AT,Austria,0,-1,0,0,0,0,-1,0,0,-1
1,75.0,751,Manifest zur Wahl des Europäischen Parlaments ...,"""Die nationalen Regierungen hintertreiben die ...",https://www.1europa.at/manifest,339,AT,Initiative 1 Europe,Initiative 1 Europe,http://files2.societly.com.s3-website.eu-centr...,,751,"{u'law_and_order': u'0', u'restrictive_financi...",None,Social programmes should be maintained even at...,AT,Austria,0,-1,0,0,0,0,-1,0,0,-1
2,100.0,751,EU-Wahlprogramm der Europapartei,"""Wir wollen ein soziales Europa mit nachhaltig...",https://vote.europeangreens.eu/manifesto,344,AT,The Greens - The Green Alternative,GRÜNE,http://files2.societly.com.s3-website.eu-centr...,,751,"{u'law_and_order': u'0', u'restrictive_financi...",None,Social programmes should be maintained even at...,AT,Austria,0,-1,0,0,0,0,-1,0,0,-1
3,25.0,751,Grundsatzprogramm 2016,"""Die Leistungen des Staates sind klar auf schl...",https://www.neos.eu/_Resources/Persistent/b0cd...,472,AT,The New Austria and Liberal Forum,NEOS,http://files2.societly.com.s3-website.eu-centr...,,751,"{u'law_and_order': u'0', u'restrictive_financi...",None,Social programmes should be maintained even at...,AT,Austria,0,-1,0,0,0,0,-1,0,0,-1
4,25.0,751,Regierungsprogramm 2017,"""Die Sozialhilfe (Mindestsicherung NEU) ist ei...",https://www.dieneuevolkspartei.at/download/Reg...,475,AT,Austrian People's Party,ÖVP,http://files2.societly.com.s3-website.eu-centr...,,751,"{u'law_and_order': u'0', u'restrictive_financi...",None,Social programmes should be maintained even at...,AT,Austria,0,-1,0,0,0,0,-1,0,0,-1


In [29]:
full = full[[
    u'answer',
    u'question_id',
    u'source',
    u'source_desc',
    u'source_link',
    'country_x',
    'name',
    'partyPositions.id',
    'partyPositions.party',
    'partyPositions.name',
    'partyPositions.image',
    u'value',
    u'question_shared_id',
    u'economic_liberalisation',
    u'environmental_protection',
    u'european_integration',
    u'law_and_order',
    u'liberal_society',
    u'restrictive_financial_policy',
    u'restrictive_immigration_policy',
    'xy-lc',
    'xy-lr'
]]

full.columns = [
    'answer',
    'question_id',
    'motivation_src',
    'motivation',
    'motivation_src_link',
    'country_code',
    'country_name',
    'party_id',
    'party_name_full',
    'party_name_short',
    'party_image',
    'question',
    'question_shared_id',
    'economic_liberalisation',
    'environmental_protection',
    'european_integration',
    'law_and_order',
    'liberal_society',
    'restrictive_financial_policy',
    'restrictive_immigration_policy',
    'xy_lc',
    'xy_lr'  
]

In [30]:
full.to_csv('../public/data/data.csv', encoding='utf-8', index=False)

In [11]:
full[full.answer.isnull()]

,answer,question_id,motivation_src,motivation,motivation_src_link,country_code,country_name,party_id,party_name_full,party_name_short,party_image,question,economic_liberalisation,environmental_protection,european_integration,law_and_order,liberal_society,restrictive_financial_policy,restrictive_immigration_policy,xy_lc,xy_lr
69,NaN,762,,,,AT,Austria,472,The New Austria and Liberal Forum,NEOS,http://files2.societly.com.s3-website.eu-centr...,Bank and stock market gains should be taxed mo...,-1,0,0,0,0,1,0,0,-1
70,NaN,762,,,,AT,Austria,475,Austrian People's Party,ÖVP,http://files2.societly.com.s3-website.eu-centr...,Bank and stock market gains should be taxed mo...,-1,0,0,0,0,1,0,0,-1
82,NaN,764,,,,AT,Austria,475,Austrian People's Party,ÖVP,http://files2.societly.com.s3-website.eu-centr...,Renewable sources of energy (e.g. solar or win...,-1,1,0,0,0,0,0,1,-1
279,NaN,794,,,,VL,Belgium - Flanders,242,Socialist Party Differently,sp.a,http://files2.societly.com.s3-website.eu-centr...,"In European Parliament elections, European Uni...",0,0,1,0,0,0,0,1,0
334,NaN,801,,,,WA,Belgium - Walloon,532,Democratic Federalist Independent,DéFI,http://files2.societly.com.s3-website.eu-centr...,The legalisation of same sex marriages is a go...,0,0,0,0,1,0,0,1,0
435,NaN,816,,,,WA,Belgium - Walloon,291,Reformist Movement,MR,http://files2.societly.com.s3-website.eu-centr...,"In European Parliament elections, European Uni...",0,0,1,0,0,0,0,1,0
436,NaN,816,,,,WA,Belgium - Walloon,292,Humanist Democratic Centre,cdH,http://files2.societly.com.s3-website.eu-centr...,"In European Parliament elections, European Uni...",0,0,1,0,0,0,0,1,0
453,NaN,818,,,,BG,Bulgaria,395,Democratic Bulgaria,ДБ,http://files2.societly.com.s3-website.eu-centr...,The state should provide stronger financial su...,-1,0,0,0,0,-1,0,0,-1
454,NaN,818,,,,BG,Bulgaria,398,Will,Воля,http://files2.societly.com.s3-website.eu-centr...,The state should provide stronger financial su...,-1,0,0,0,0,-1,0,0,-1
459,NaN,819,,,,BG,Bulgaria,385,Bulgarian Socialist Party,БСП,http://files2.societly.com.s3-website.eu-centr...,The European Union should rigorously punish Me...,0,0,1,0,0,1,0,1,0


In [384]:
full[full.party_id == 386][['question', 'answer', 'european_integration']]

,question,answer,european_integration
2910,Social programmes should be maintained even at...,100.0,0
2920,The state should provide stronger financial su...,100.0,0
2930,The European Union should rigorously punish Me...,75.0,1
2940,Asylum-seekers should be distributed proportio...,NaN,1
2950,Immigration into Ireland should be made more r...,NaN,0
2960,Immigrants from outside Europe should be requi...,NaN,0
2970,The legalisation of same sex marriages is a go...,100.0,0
2980,The legalisation of the personal use of soft d...,NaN,0
2990,Euthanasia should be legalised,NaN,0
3000,Government spending should be reduced in order...,0.0,0


In [41]:
full[(full.country_code == 'NL') & (full['party_name_short'] == 'VVD') & (full['answer'] == 100)]

,answer,question_id,motivation_src,motivation,motivation_src_link,country_code,country_name,party_id,party_name_full,party_name_short,party_image,question,question_shared_id,economic_liberalisation,environmental_protection,european_integration,law_and_order,liberal_society,restrictive_financial_policy,restrictive_immigration_policy,xy_lc,xy_lr
3962,100.0,1193,EP Verkiezingsprogramma 2019,We maken scherpe afspraken en dwingen deze af....,https://www.vvd.nl/content/uploads/2016/11/Ver...,NL,Netherlands,247,People's Party for Freedom and Democracy,VVD,http://files2.societly.com.s3-website.eu-centr...,The European Union should rigorously punish Me...,2,0,0,1,0,0,1,0,1,0
3986,100.0,1195,EP Verkiezingsprogramma 2019,De VVD wil grip op migratie en een einde maken...,https://www.vvd.nl/content/uploads/2016/11/Ver...,NL,Netherlands,247,People's Party for Freedom and Democracy,VVD,http://files2.societly.com.s3-website.eu-centr...,Immigration into the Netherlands should be mad...,4,0,0,0,1,-1,0,1,-1,0
3998,100.0,1196,TK Verkiezingsprogramma 2017,Mensen die zich tegen onze vrijheden en onze w...,https://www.vvd.nl/content/uploads/2016/11/2_v...,NL,Netherlands,247,People's Party for Freedom and Democracy,VVD,http://files2.societly.com.s3-website.eu-centr...,Immigrants from outside Europe should be requi...,5,0,0,0,0,0,0,1,-1,0
4010,100.0,1197,TK Verkiezingsprogramma 2017,En we vinden dat allemaal heel gewoon. Net zoa...,https://www.vvd.nl/content/uploads/2016/11/2_v...,NL,Netherlands,247,People's Party for Freedom and Democracy,VVD,http://files2.societly.com.s3-website.eu-centr...,The legalisation of same sex marriages is a go...,6,0,0,0,0,1,0,0,1,0
4034,100.0,1199,TK Verkiezingsprogramma 2017,De mogelijkheden hiervoor binnen de euthanasie...,https://www.vvd.nl/content/uploads/2016/11/2_v...,NL,Netherlands,247,People's Party for Freedom and Democracy,VVD,http://files2.societly.com.s3-website.eu-centr...,Euthanasia should be legalised,8,0,0,0,-1,1,0,0,1,0
4118,100.0,1206,TK Verkiezingsprogramma 2017,Wij willen daarom hogere maximale straffen voo...,https://www.vvd.nl/content/uploads/2016/11/2_v...,NL,Netherlands,247,People's Party for Freedom and Democracy,VVD,http://files2.societly.com.s3-website.eu-centr...,Criminals should be punished more severely,15,0,0,0,1,-1,0,0,-1,0


In [40]:
full[full.country_code == 'NL'].groupby(['party_name_short', 'answer']).count()

question_id  motivation_src  motivation  \
party_name_short answer                                            
50+              0.0               4               4           4   
                 25.0              1               1           1   
                 75.0              3               3           3   
                 100.0            13              13          13   
CDA              0.0               2               2           2   
                 25.0              5               5           5   
                 50.0              6               6           6   
                 75.0              5               5           5   
                 100.0             4               4           4   
CU-SGP           0.0               5               5           5   
                 25.0              5               5           5   
                 50.0              4               4           4   
                 75.0              8               8           8   
D66              0.0               4               4           4   
                 25.0              1               1           1   
                 50.0              2               2           2   
                 75.0              4               4           4   
                 100.0            11              11          11   
DENK             0.0               4               4           4   
                 25.0              3               3           3   
                 75.0              2               2           2   
                 100.0             6               6           6   
FvD              0.0               5               5           5   
                 25.0              3               3           3   
                 50.0              1               1           1   
                 100.0             8               8           8   
GL               25.0              6               6           6   
                 50.0              1               1           1   
                 75.0              9               9           9   
                 100.0             6               6           6   
PVV              0.0               9               9           9   
                 25.0              3               3           3   
                 50.0              1               1           1   
                 75.0              3               3           3   
                 100.0             5               5           5   
PvdA             25.0              5               5           5   
                 50.0              3               3           3   
                 75.0              7               7           7   
                 100.0             7               7           7   
PvdD             0.0               3               3           3   
                 25.0              5               5           5   
                 50.0              3               3           3   
                 75.0              9               9           9   
                 100.0             2               2           2   
SP               0.0               3               3           3   
                 25.0              8               8           8   
                 50.0              1               1           1   
                 75.0              2               2           2   
                 100.0             8               8           8   
VVD              0.0               1               1           1   
                 25.0              8               8           8   
                 50.0              2               2           2   
                 75.0              4               4           4   
                 100.0             6               6           6   

                         motivation_src_link  country_code  country_name  \
party_name_short answer                                                    
50+              0.0                       4             4      